In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
.appName('Caching in big files').enableHiveSupport().getOrCreate()

25/06/18 06:33:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
!hadoop fs -ls /data/

Found 5 items
-rw-r--r--   2 root               hadoop       5488 2025-06-02 15:26 /data/customers_100.csv
-rw-r--r--   2 geurecruitanandraj hadoop   10528211 2025-06-17 15:58 /data/customers_10mb.csv
-rw-r--r--   2 geurecruitanandraj hadoop  168541068 2025-06-17 15:56 /data/customers_150mb.csv
-rw-r--r--   2 geurecruitanandraj hadoop    8628599 2025-06-17 15:58 /data/orders_10.csv
-rw-r--r--   2 geurecruitanandraj hadoop    7896035 2025-06-17 15:59 /data/shippings_10.csv


In [4]:
cust_rdd = spark.sparkContext.textFile('/data/customers_150mb.csv')

In [5]:
cust_rdd.top(5)

['customer_id,name,city,state,country,registration_date,is_active',
 '999999,Customer_999999,Bangalore,Karnataka,India,2023-10-22,True',
 '999998,Customer_999998,Mumbai,Gujarat,India,2023-07-14,True',
 '999997,Customer_999997,Hyderabad,Gujarat,India,2023-07-26,False',
 '999996,Customer_999996,Hyderabad,Telangana,India,2023-03-17,False']

In [6]:
cust_filtered = cust_rdd.filter(lambda row:'Mumbai' in row)
cust_map = cust_filtered.map(lambda row:(row.split(',')[0],1))
cust_reduced = cust_map.reduceByKey(lambda x,y:x+y)

In [7]:
import time
start_time = time.time()
cust_reduced.count()
end_time = time.time()
print(f"Time taken : {end_time - start_time} seconds")

Time taken : 5.322003364562988 seconds


In [8]:
start_time = time.time()
cust_reduced.count()
end_time = time.time()
print(f"Time taken : {end_time - start_time} seconds")

Time taken : 0.37826013565063477 seconds


In [9]:
cust_reduced.cache()

PythonRDD[9] at RDD at PythonRDD.scala:53

In [10]:
# First time it would take more time, as caching is lazy , it would take time in caching as well.
start_time = time.time()
cust_reduced.count()
end_time = time.time()
print(f"Time taken : {end_time - start_time} seconds")

Time taken : 0.7448794841766357 seconds


In [11]:
# Second run would be much more faster
start_time = time.time()
cust_reduced.count()
end_time = time.time()
print(f"Time taken : {end_time - start_time} seconds")

Time taken : 0.24593353271484375 seconds
